# Introduction

In this tutorial, we will demonstrate how to use **Enclaves** in PySyft to securely perform computations among assets from multiple domains. We will cover the following workflows:

Constraints:

1. Once the enclave is set up for multiparty code execution, no domain has special privileges, and all domain are the same citizen, even the one responsible for leasing the Enclave.
2. Domains must not communicate with each other, neither should the system require domains to be aware of the presence of other domains.
3. System should be able to run the computation on both long-running Enclaves, as well as ephemeral (on-demand) Enclaves.
4. DS is assumed to have the highest interest in ensuring code execution.
5. DS is aware of all the domains and enclaves present in the network.
6. Principle of least privilege is followed.

## Data Owners Workflow - Part 1
[./00-do-setup-domain.ipynb](./00-do-setup-domain.ipynb)
- Launch two domain servers for providing data to the data scientist.
- Launch one enclave server for performing the secure computation using data from both the domain servers.
- Upload datasets to both the domain servers.
- Register an account for the data scientist in both the domain servers.
- Register the enclave server with one of the domain servers for discoverability by the data scientist.

## Data Scientist Workflow - Part 1
[./01-ds-submit-project.ipynb](./01-ds-submit-project.ipynb)
- Find datasets across multiple domains.
- Find a suitable Enclave for performing the multi-party computation.
- Create a project containing code to perform multi-party computation.
- Submit the project for review by the data owners.

## Data Owner Workflow - Part 2
[./02-do-review-code.ipynb](./02-do-review-code.ipynb)
- View all pending projects.
- Select a project and review the code, assets and the policies.
- Run the code on mock data from all the dependent domains.
- Approve the request.

## Data Scientist Workflow - Part 2
[./03-ds-execute-code.ipynb](./03-ds-execute-code.ipynb)
- Check the project approval.
- Request execution when the code is approved and ready
- Download the result.

In [ ]:
# third party
import pytest
from recordlinkage.datasets import load_febrl4

# syft absolute
import syft as sy
from syft.abstract_server import ServerType
from syft.service.network.routes import HTTPServerRoute
from syft.service.response import SyftAttributeError
from syft.service.response import SyftSuccess

CANADA_DOMAIN_PORT = 9081
ITALY_DOMAIN_PORT = 9082
CANADA_ENCLAVE_PORT = 9083

# Launch servers

We will begin by launching two domain servers and an enclave server.

In [ ]:
canada_server = sy.orchestra.launch(
    name="canada-domain", port=CANADA_DOMAIN_PORT, dev_mode=True, reset=True
)
italy_server = sy.orchestra.launch(
    name="italy-domain", port=ITALY_DOMAIN_PORT, dev_mode=True, reset=True
)
canada_enclave = sy.orchestra.launch(
    name="canada-enclave",
    server_type=ServerType.ENCLAVE,
    port=CANADA_ENCLAVE_PORT,
    dev_mode=True,
    reset=True,
)

In [ ]:
do_canada_client = canada_server.login(
    email="info@openmined.org", password="changethis"
)
do_italy_client = italy_server.login(email="info@openmined.org", password="changethis")

assert do_canada_client.metadata.server_type == ServerType.DOMAIN
assert do_italy_client.metadata.server_type == ServerType.DOMAIN

# Upload datasets to both domains

In [ ]:
# Using public datasets from Freely Extensible Biomedical Record Linkage (Febrl) project
canada_census_data, italy_census_data = load_febrl4()

In [ ]:
for dataset, client, country in zip(
    [canada_census_data, italy_census_data],
    [do_canada_client, do_italy_client],
    ["Canada", "Italy"],
):
    private_data, mock_data = dataset[:2500], dataset[2500:]
    dataset = sy.Dataset(
        name=f"{country} - FEBrl Census Data",
        description="abc",
        asset_list=[
            sy.Asset(
                name="census_data",
                mock=mock_data,
                data=private_data,
                shape=private_data.shape,
                mock_is_real=True,
            )
        ],
    )
    client.upload_dataset(dataset)

In [ ]:
assert len(do_canada_client.datasets.get_all()) == 1
assert len(do_italy_client.datasets.get_all()) == 1

# Create account for data scientist on both the domains

In [ ]:
for client in [do_canada_client, do_italy_client]:
    res = client.register(
        name="Sheldon",
        email="sheldon@caltech.edu",
        password="changethis",
        password_verify="changethis",
    )
    assert isinstance(res, SyftSuccess)

# Register the enclave with Canada domain

In [ ]:
route = HTTPServerRoute(host_or_ip="localhost", port=CANADA_ENCLAVE_PORT)
do_canada_client.enclaves.add(route=route)

In [ ]:
assert (len(do_canada_client.enclaves.get_all())) == 1
do_canada_client.enclaves.get_all()

In [ ]:
ds_canada_client = sy.login(
    email="sheldon@caltech.edu", password="changethis", port=CANADA_DOMAIN_PORT
)

In [ ]:
# Data scientist should not be able to add enclave to the domain
with pytest.raises(SyftAttributeError) as exc_info:
    ds_canada_client.enclaves.add(
        name="Dummy Enclave", route=HTTPServerRoute(host_or_ip="localhost", port=9084)
    )
print(exc_info.value)

In [ ]:
# Ensure that the data scientist can access the enclave added by the domain owner
assert (len(ds_canada_client.enclaves.get_all())) == 1
ds_canada_client.enclaves.get_all()

# Cleanup local domain servers

In [ ]:
if canada_server.deployment_type.value == "python":
    canada_server.land()

if italy_server.deployment_type.value == "python":
    italy_server.land()

if canada_enclave.deployment_type.value == "python":
    canada_enclave.land()